In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import re
from datetime import datetime


In [112]:
def get_one_bank_link(bank_frame):
    return "http://bankodrom.ru" + bank_frame.a["href"]
def get_all_banks_from_one_page(all_bank_frames):
    return pd.Series(all_bank_frames, dtype = "object").apply(get_one_bank_link)

class sample_strings():
    
    def __init__(self):
        
        html_one_bank = requests.get("http://www.bankodrom.ru/bank/sberbank/").text
        soup_one_bank = bs4.BeautifulSoup(html_one_bank, "html.parser")
        self.region_text = soup_one_bank.find_all("th")[7].text
        
        one_rating_url = "http://bankodrom.ru/bank/absolyut-bank/" + "kreditnye-rejtingi/"
        html_one_bank = requests.get(one_rating_url).text
        soup_one_bank = bs4.BeautifulSoup(html_one_bank, "html.parser")
        self.moodys_template = soup_one_bank.find_all("h2")[2].string
        self.expertra_template = soup_one_bank.find_all("h2")[0].string
        
    def moodys_text(self, bank):
        return re.sub("Абсолют банк", bank, self.moodys_template)
    
    def expertra_text(self, bank):
        return re.sub("Абсолют банк", bank, self.expertra_template)
    
def get_one_bank_data(one_bank_url, samples):
    try:
        html_one_bank = requests.get(one_bank_url).text
    except:
        try:
            html_one_bank = requests.get(one_bank_url).text
        except:
                return {"short_name" : "Error", "form" : "Error", "regn" : "Error",
            "region" : "Error", "ownership" : "Error", 
            "personal_page" : "Error", "app" : "Error"}
    soup_one_bank = bs4.BeautifulSoup(html_one_bank, "html.parser")
    try:
        short_name = soup_one_bank.find("th", string = "Сокращенное наименование").next_sibling.next_sibling.text
    except:
        short_name = "no"
    try:
        form = soup_one_bank.find("th", string = "Организационно-правовая форма").next_sibling.next_sibling.text
    except:
        form = "no"
    try:
        regn = soup_one_bank.find("th", string = "Лицензия ЦБ").next_sibling.next_sibling.text
    except:
        regn = "no"
    region_text = samples.region_text
    try:
        region = soup_one_bank.find("th", string = region_text).next_sibling.next_sibling.text
    except:
        region = "no"
    try:
        ownership = soup_one_bank.find("th", string = "Тип собственности").next_sibling.next_sibling.text
        ownership = re.sub("\n","",ownership)
    except:
        ownership = "no"
    if soup_one_bank.find_all("th", string = "Личный кабинет") == []:
        personal_page = 0
    else:
        personal_page = 1
    if (soup_one_bank.find_all("th", string = "Мобильное приложение (iOs)") == [])&\
        (soup_one_bank.find_all("th", string = "Мобильное приложение (Android)") == []):
        app = 0
    else:
        app = 1
    return {"short_name" : short_name, "form" : form, "regn" : regn,
            "region" : region, "ownership" : ownership, 
            "personal_page" : personal_page, "app" : app}

def get_one_bank_credit_rating(one_bank_short_name_and_url, samples):
    print(one_bank_short_name_and_url[0])
    one_rating_url = one_bank_short_name_and_url[1] + "kreditnye-rejtingi/"
    short_name = one_bank_short_name_and_url[0]
    try:
        html_one_bank = requests.get(one_rating_url).text
    except:
        try:
            html_one_bank = requests.get(one_rating_url).text
        except:
            thetablera = pd.DataFrame({"DT" : "no", "Moodys" : "no", "name" : short_name}, index = [0])
            thetablem = pd.DataFrame({"DT" : "no", "Moodys" : "no", "name" : short_name}, index = [0])
            return pd.concat([thetablem, thetablera])
    soup_one_bank = bs4.BeautifulSoup(html_one_bank, "html.parser")
    try:
        thetablem = soup_one_bank.find("h2", string = samples.moodys_text(short_name)).next_sibling.next_sibling
        thetablem = pd.read_html(str(thetablem))[0]
        thetablem = thetablem[["Дата", "Долгосрочный,нац. валюта"]]
        thetablem["name"] = short_name

    except AttributeError:
        thetablem = pd.DataFrame({"DT" : "no", "Moodys" : "no", "name" : short_name}, index = [0])
    thetablem["ratingname"] = "Moodys"
    thetablem.columns = ["DT", "rating", "name", "ratingname"]
      
    try:
        thetablera = soup_one_bank.find("h2", string = samples.expertra_text(short_name)).next_sibling.next_sibling
        thetablera = pd.read_html(str(thetablera))[0]
        thetablera = thetablera[["Дата", "Национальнаяшкала"]]
        thetablera["name"] = short_name
    except AttributeError:
        thetablera = pd.DataFrame({"DT" : "no", "Moodys" : "no", "name" : short_name}, index = [0])
    thetablera["ratingname"] = "Expertra"
    thetablera.columns = ["DT", "rating", "name", "ratingname"]

    return pd.concat([thetablem, thetablera])
    #return thetablem

### Get links to banks

In [88]:
#for one page
url_banklist = "https://www.bankodrom.ru/banki-rossii/?p=1"
html_banklist = requests.get(url_banklist).text
soup_banklist = bs4.BeautifulSoup(html_banklist, "html.parser")
all_bank_frames = soup_banklist.find_all("td", {"class" : "bank-info-descr"})
#get_all_banks_from_one_page(all_bank_frames)

In [89]:
#for all pages
all_banks_links = pd.Series([], dtype = "object")
for i in range(1,10):
    url_banklist = f"https://www.bankodrom.ru/banki-rossii/?p={i}"
    html_banklist = requests.get(url_banklist).text
    soup_banklist = bs4.BeautifulSoup(html_banklist, "html.parser")
    all_bank_frames = soup_banklist.find_all("td", {"class" : "bank-info-descr"})
    this_page_links = get_all_banks_from_one_page(all_bank_frames)
    all_banks_links = all_banks_links.append(this_page_links, ignore_index = True)

In [90]:
all_banks_links

0        http://bankodrom.ru/bank/absolyut-bank/
1             http://bankodrom.ru/bank/avangard/
2                http://bankodrom.ru/bank/avers/
3         http://bankodrom.ru/bank/avtogradbank/
4       http://bankodrom.ru/bank/avtokreditbank/
                         ...                    
434          http://bankodrom.ru/bank/yumk-bank/
435    http://bankodrom.ru/bank/yunikredit-bank/
436          http://bankodrom.ru/bank/yunistrim/
437      http://bankodrom.ru/bank/yandeks-dengi/
438       http://bankodrom.ru/bank/yarinterbank/
Length: 439, dtype: object

### Get banks data

In [91]:
#for one bank
one_bank_url = "https://www.bankodrom.ru/bank/vega-bank/"
html_one_bank = requests.get(one_bank_url).text
soup_one_bank = bs4.BeautifulSoup(html_one_bank, "html.parser")
try:
    short_name = soup_one_bank.find("th", string = "Сокращенное наименование").next_sibling.next_sibling.text
except:
    short_name = "no"
try:
    form = soup_one_bank.find("th", string = "Организационно-правовая форма").next_sibling.next_sibling.text
except:
    form = "no"
try:
    regn = soup_one_bank.find("th", string = "Лицензия ЦБ").next_sibling.next_sibling.text
except:
    regn = "no"
region_text = samples.region_text
try:
    region = soup_one_bank.find("th", string = region_text).next_sibling.next_sibling.text
except:
    region = "no"
try:
    ownership = soup_one_bank.find("th", string = "Тип собственности").next_sibling.next_sibling.text
    ownership = re.sub("\n","",ownership)
except:
    ownership = "no"
if soup_one_bank.find_all("th", string = "Личный кабинет") == []:
    personal_page = 0
else:
    personal_page = 1
if (soup_one_bank.find_all("th", string = "Мобильное приложение (iOs)") == [])&\
    (soup_one_bank.find_all("th", string = "Мобильное приложение (Android)") == []):
    app = 0
else:
    app = 1

print( {"short_name" : short_name, "form" : form, "regn" : regn,
        "region" : region, "ownership" : ownership, 
        "personal_page" : personal_page, "app" : app})


{'short_name': 'Вега-банк', 'form': 'Общество с ограниченной ответственностью (ООО)', 'regn': '3270', 'region': 'Москва', 'ownership': 'no', 'personal_page': 0, 'app': 0}


In [92]:
#for all banks
samples = sample_strings()
all_banks_data = all_banks_links.apply(get_one_bank_data, samples = samples)
all_banks_data = pd.DataFrame(list(all_banks_data))

In [93]:
all_banks_data

,short_name,form,regn,region,ownership,personal_page,app
0,Абсолют банк,Публичное акционерное общество (ПАО),2306,Москва,Частный банк,1,1
1,Авангард,Публичное акционерное общество (ПАО),2879,Москва,Частный банк,1,1
2,Аверс,Общество с ограниченной ответственностью (ООО),415,Республика Татарстан (Татарстан),Частный банк,1,1
3,Автоградбанк,Акционерное общество (АО),1455,Республика Татарстан (Татарстан),Частный банк,1,1
4,Автокредитбанк,Общество с ограниченной ответственностью (ООО),1973,Республика Татарстан (Татарстан),Частный банк,0,0
...,...,...,...,...,...,...,...
434,ЮМК банк,Общество с ограниченной ответственностью (ООО),3495,Краснодарский край,Частный банк,0,0
435,ЮниКредит Банк,Акционерное общество (АО),1,Москва,Банк с иностранным капиталом,1,1
436,Юнистрим,Акционерное общество (АО),3467,Москва,Частный банк,0,1
437,Яндекс.Деньги,Общество с ограниченной ответственностью (ООО),3510-К,Москва,Государственный банк,0,0


### Get credit ratings

In [100]:
#for one bank
samples = sample_strings()
one_rating_url = "http://bankodrom.ru/bank/sberbank/" + "kreditnye-rejtingi/"
short_name = "Сбербанк"
html_one_bank = requests.get(one_rating_url).text
soup_one_bank = bs4.BeautifulSoup(html_one_bank, "html.parser")

thetablem = soup_one_bank.find("h2", string = samples.moodys_text(short_name)).next_sibling.next_sibling
thetablem = pd.read_html(str(thetablem))[0]
thetablem = thetablem[["Дата", "Долгосрочный,нац. валюта"]]
thetablem["name"] = short_name
thetablem

thetablera = soup_one_bank.find("h2", string = samples.expertra_text(short_name)).next_sibling.next_sibling
thetablera = pd.read_html(str(thetablera))[0]
thetablera = thetablera[["Дата", "Национальнаяшкала"]]
thetablera["name"] = short_name
thetablera


            Сбербанк, кредитный рейтинг Эксперт РА
        


AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [101]:
#for all banks
all_banks_names_urls = tuple(zip(all_banks_data.short_name, all_banks_links))
all_banks_moodys_list = list(pd.Series(all_banks_names_urls).apply(get_one_bank_credit_rating, samples = samples))
all_banks_moodys_table = pd.concat(all_banks_moodys_list, ignore_index = True)

Абсолют банк
Авангард
Аверс
Автоградбанк
Автокредитбанк
Автоторгбанк
Агропромкредит
Агророс
Азиатско-Тихоокеанский банк (АТБ)
Азия-Инвест Банк
АйСиБиСи Банк
АК БАРС
Акибанк
Акрополь
Актив банк
Александровский
Алеф-банк
Алмазэргиэнбанк (АЭБ)
Алтайкапиталбанк
Алтынбанк
Альба Альянс
Альтернатива
Альфа-банк
Америкэн Экспресс Банк
Апабанк
Аресбанк
Арзамас
Байкалинвестбанк
Байкалкредобанк
Балаково-банк
Балтинвестбанк
Банк 131
Банк Агора
Банк Акцепт
Банк БЖФ
Банк БКФ
Банк ДОМ.РФ
Банк Интеза
Банк Казани
Банк Кредит Свисс
Банк НБС
Банк Оранжевый
Банк Оренбург
Банк ПСА Финанс РУС
Банк развития и модернизации промышленности
Банк Развития Русской Сети Интернет (РСИ)
Банк Реалист
Банк Сберегательно-кредитного сервиса
Банк СГБ
ББР Банк
Белгородсоцбанк
Берейт
Бест Эффортс Банк
Бизнес-Сервис-Траст
БКС Банк
БМ-Банк (бывш. Банк Москвы)
БМВ Банк
БНП Париба Банк
Братский
БыстроБанк
Бэнк Оф Чайна
Вакобанк
Великие Луки
Венец
Веста
Вестерн Юнион ДП Восток
Викинг
Витабанк
ВКАБанк
Владбизнесбанк
Внешфинбанк
Во

In [104]:
all_banks_moodys_table

,DT,rating,name,ratingname
0,26.11.2019,B2,Абсолют банк,Moodys
1,29.03.2019,B2,Абсолют банк,Moodys
2,21.11.2018,B2,Абсолют банк,Moodys
3,26.04.2016,B1,Абсолют банк,Moodys
4,18.03.2016,B1,Абсолют банк,Moodys
...,...,...,...,...
2764,24.02.2014,A+ (II),Юнистрим,Expertra
2765,no,no,Яндекс.Деньги,Moodys
2766,no,no,Яндекс.Деньги,Expertra
2767,no,no,Яринтербанк,Moodys


### Переформатируем даты в таблице с рейтингами

In [106]:
dates_template = pd.DataFrame({"DateMonthly" : pd.date_range('01.01.2015','12.01.2020', 
              freq='MS').strftime("%d.%m.%Y").tolist()})
dates_template["DateMonthly"] = dates_template.DateMonthly.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%d.%m.%Y").date())

banks_df = all_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Expertra")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
expertra_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = expertra_table.append(one_bank_filled_merge)
expertra_table = pd.concat(expertra_table, ignore_index = True)
expertra_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
expertra_table.columns = ["DT", "ExpertRA", "name"]

banks_df = all_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Expertra")]
no_expertra_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Expertra"] = "no"
    dates_template_local["name"] = bank_name
    no_expertra_table.append(dates_template_local)
no_expertra_table = pd.concat(no_expertra_table, ignore_index = True)
no_expertra_table.columns = ["DT", "ExpertRA", "name"]
expertra_table = pd.concat([expertra_table, no_expertra_table], ignore_index = True)
                                   

banks_df = all_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Moodys")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
moodys_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = moodys_table.append(one_bank_filled_merge)
moodys_table = pd.concat(moodys_table, ignore_index = True)
moodys_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
moodys_table.columns = ["DT", "Moodys", "name"]

banks_df = all_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Moodys")]
no_moodys_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Moodys"] = "no"
    dates_template_local["name"] = bank_name
    no_moodys_table.append(dates_template_local)
no_moodys_table = pd.concat(no_moodys_table, ignore_index = True)
no_moodys_table.columns = ["DT", "Moodys", "name"]
moodys_table = pd.concat([moodys_table, no_moodys_table], ignore_index = True)



ratings_table = moodys_table.merge(expertra_table, on = ["DT", "name"], how = "outer")

In [107]:
ratings_table

,DT,Moodys,name,ExpertRA
0,2015-01-01,NaN,Абсолют банк,NaN
1,2015-02-01,NaN,Абсолют банк,A++
2,2015-03-01,B1,Абсолют банк,A++
3,2015-04-01,B1,Абсолют банк,A++
4,2015-05-01,B1,Абсолют банк,A++
...,...,...,...,...
31628,2020-08-01,no,Яринтербанк,no
31629,2020-09-01,no,Яринтербанк,no
31630,2020-10-01,no,Яринтербанк,no
31631,2020-11-01,no,Яринтербанк,no


### Объединим собранные таблицы

In [108]:
result_table = ratings_table.merge(all_banks_data, left_on = "name", right_on = "short_name", how = "outer")

In [109]:
result_table.fillna("no", inplace = True)

## Now repeat the same procedure for banks with withdrawn licences

In [113]:
all_withdrawn_banks_links = pd.Series([], dtype = "object")
for i in range(1,13):
    url_banklist = f"https://www.bankodrom.ru/spisok-bankov-u-kotoryh-otozvali-licenziyu/?p={i}"
    html_banklist = requests.get(url_banklist).text
    soup_banklist = bs4.BeautifulSoup(html_banklist, "html.parser")
    all_bank_frames = soup_banklist.find_all("td", {"class" : "bank-info-descr"})
    this_page_links = get_all_banks_from_one_page(all_bank_frames)
    all_withdrawn_banks_links = all_withdrawn_banks_links.append(this_page_links, ignore_index = True)
    
samples = sample_strings()
all_withdrawn_banks_data = all_withdrawn_banks_links.apply(get_one_bank_data, samples = samples)
all_withdrawn_banks_data = pd.DataFrame(list(all_withdrawn_banks_data))

#for all banks
all_withdrawn_banks_names_urls = tuple(zip(all_withdrawn_banks_data.short_name, all_withdrawn_banks_links))
all_withdrawn_banks_moodys_list = list(pd.Series(all_withdrawn_banks_names_urls).apply(get_one_bank_credit_rating, samples = samples))
all_withdrawn_banks_moodys_table = pd.concat(all_withdrawn_banks_moodys_list, ignore_index = True)



1банк
21 век
АББ
Агроинкомбанк
Агросоюз
Агрохимбанк
Адам Интернэшнл
Аделантбанк
Адмиралтейский
Азимут
Азия Банк
Айви Банк
АйМаниБанк
АкадемРусБанк
Аксонбанк
АктивКапитал Банк
Акцент
Акция
Алданзолотобанк
Алжан
Алтайбизнес-банк
Альта-банк
АМБ Банк
АМТ банк
Анджибанк
Анелик РУ
Анкор Банк Сбережений
Анталбанк
АРБ-ИНКАСС
Арксбанк
Арсенал
Арт-банк
Аскания траст
Аскольд
Аспект
Ассигнация
Ассоциация
Атлас банк
Ауэрбанк
АФ банк
Байкалбанк
Балтика
Балткредобанк
Банк - Т
Банк БФТ
Банк ВЕФК-Урал
Банк Высоких Технологий
Банк инноваций и развития
Банк МБСП
Банк на Красных Воротах
Банк Проектного Финансирования
Банк Развития Бизнеса
Банк развития инвестиций и технологий (Центр)
Банк Развития Предпринимательства
Банк Развития Региона
Банк Развития Технологий
Банк Расчетов и Сбережений
Банк Торгового Финансирования
Банк24.ру
Банкирский Дом
Башкомснаббанк
БВА банк
Бенифит-банк
Бизнес для Бизнеса
Бизнесбанк
Богородский
Богородский муниципальный банк
Борский Коммерческий Банк
Бузулукбанк
Булгар банк
Бум-

In [114]:
dates_template = pd.DataFrame({"DateMonthly" : pd.date_range('01.01.2015','12.01.2020', 
              freq='MS').strftime("%d.%m.%Y").tolist()})
dates_template["DateMonthly"] = dates_template.DateMonthly.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%d.%m.%Y").date())

banks_df = all_withdrawn_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Expertra")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
expertra_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = expertra_table.append(one_bank_filled_merge)
expertra_table = pd.concat(expertra_table, ignore_index = True)
expertra_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
expertra_table.columns = ["DT", "ExpertRA", "name"]

banks_df = all_withdrawn_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Expertra")]
no_expertra_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Expertra"] = "no"
    dates_template_local["name"] = bank_name
    no_expertra_table.append(dates_template_local)
no_expertra_table = pd.concat(no_expertra_table, ignore_index = True)
no_expertra_table.columns = ["DT", "ExpertRA", "name"]
expertra_table = pd.concat([expertra_table, no_expertra_table], ignore_index = True)
                                   

banks_df = all_withdrawn_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Moodys")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
moodys_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = moodys_table.append(one_bank_filled_merge)
moodys_table = pd.concat(moodys_table, ignore_index = True)
moodys_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
moodys_table.columns = ["DT", "Moodys", "name"]

banks_df = all_withdrawn_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Moodys")]
no_moodys_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Moodys"] = "no"
    dates_template_local["name"] = bank_name
    no_moodys_table.append(dates_template_local)
no_moodys_table = pd.concat(no_moodys_table, ignore_index = True)
no_moodys_table.columns = ["DT", "Moodys", "name"]
moodys_table = pd.concat([moodys_table, no_moodys_table], ignore_index = True)



withdrawn_ratings_table = moodys_table.merge(expertra_table, on = ["DT", "name"], how = "outer")
withdrawn_results_table = withdrawn_ratings_table.merge(all_withdrawn_banks_data, left_on = "name", right_on = "short_name", how = "outer")

In [115]:
withdrawn_results_table

,DT,Moodys,name,ExpertRA,short_name,form,regn,region,ownership,personal_page,app
0,2015-01-01,NaN,АйМаниБанк,no,АйМаниБанк,Общество с ограниченной ответственностью (ООО),1975,Москва,no,0,0
1,2015-02-01,NaN,АйМаниБанк,no,АйМаниБанк,Общество с ограниченной ответственностью (ООО),1975,Москва,no,0,0
2,2015-03-01,NaN,АйМаниБанк,no,АйМаниБанк,Общество с ограниченной ответственностью (ООО),1975,Москва,no,0,0
3,2015-04-01,NaN,АйМаниБанк,no,АйМаниБанк,Общество с ограниченной ответственностью (ООО),1975,Москва,no,0,0
4,2015-05-01,NaN,АйМаниБанк,no,АйМаниБанк,Общество с ограниченной ответственностью (ООО),1975,Москва,no,0,0
...,...,...,...,...,...,...,...,...,...,...,...
40226,2020-08-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0
40227,2020-09-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0
40228,2020-10-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0
40229,2020-11-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0


## И для ликвидированных банков

In [116]:
all_liquidated_banks_links = pd.Series([], dtype = "object")
for i in range(1,45):
    url_banklist = f"https://www.bankodrom.ru/spisok-likvidirovannyh-bankov/?status=5&p={i}"
    html_banklist = requests.get(url_banklist).text
    soup_banklist = bs4.BeautifulSoup(html_banklist, "html.parser")
    all_bank_frames = soup_banklist.find_all("td", {"class" : "bank-info-descr"})
    this_page_links = get_all_banks_from_one_page(all_bank_frames)
    all_liquidated_banks_links = all_liquidated_banks_links.append(this_page_links, ignore_index = True)
    
samples = sample_strings()
all_liquidated_banks_data = all_liquidated_banks_links.apply(get_one_bank_data, samples = samples)
all_liquidated_banks_data = pd.DataFrame(list(all_liquidated_banks_data))

#for all banks
all_liquidated_banks_names_urls = tuple(zip(all_liquidated_banks_data.short_name, all_liquidated_banks_links))
all_liquidated_banks_moodys_list = list(pd.Series(all_liquidated_banks_names_urls).apply(get_one_bank_credit_rating, samples = samples))
all_liquidated_banks_moodys_table = pd.concat(all_liquidated_banks_moodys_list, ignore_index = True)



АБ Финанс Банк
Абаканинвестбанк
Абдулиносельхозбанк
АБИ-банк
Абрис
АВ
Авалон
Авантаж
Августокоммерцбанк
Авиабанк
Авиастарбанк
Аврора
Автобанк-Никойл
Автовазбанк
Автогазбанк
Автодор
Автосельхозмаш
АвтоУАЗбанк
Автроконбанк
Авуар
Агат
Агвали
Агентство Кредитных Гарантий
Агидель
Агинский
Аграрник
Агробизнес
Агродорбанк
Агроком
Агрокредит
Агромашбанк
Агропромбанк
Агропромбанк
Агропромкоопбанк
Агрос-банк
Агросервис
Агросодружество
АгроТЕСбанк
Агротехинвестбанк
Агротехника
Агротехпромбанк
Агротэк
Агроуниверсалбанк
Агроформбанк
Агсбанк
АГУЛ-банк
Адамас-Самара-Банк
Адамовка
Адмирал
Азалбанк
Азвесь
Азия-Траст
Азово-Черноморский
АИГ Инвестиционный банк
АИР
Аирбанк
АКА Банк
Академический банк
Академстройбанк
АКБП
Аквилон
Акида
Аккор-банк
Акобанк
АКОбанк-Волга
Акомбанк
Акомбанк
Акпарс
Акрабанк
Аксайагро
Аксбанк
Аксион
Актив
Актисбанк
Акуша
Акцептбанк
Акционерный народный банк
Акционерный Сибирский промышленный банк
Акционерный торгово-коммерческий банк
Акционерный торгово-промышленный  банк
Алатырс

In [117]:
dates_template = pd.DataFrame({"DateMonthly" : pd.date_range('01.01.2015','12.01.2020', 
              freq='MS').strftime("%d.%m.%Y").tolist()})
dates_template["DateMonthly"] = dates_template.DateMonthly.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%d.%m.%Y").date())

banks_df = all_liquidated_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Expertra")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
expertra_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = expertra_table.append(one_bank_filled_merge)
expertra_table = pd.concat(expertra_table, ignore_index = True)
expertra_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
expertra_table.columns = ["DT", "ExpertRA", "name"]

banks_df = all_liquidated_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Expertra")]
no_expertra_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Expertra"] = "no"
    dates_template_local["name"] = bank_name
    no_expertra_table.append(dates_template_local)
no_expertra_table = pd.concat(no_expertra_table, ignore_index = True)
no_expertra_table.columns = ["DT", "ExpertRA", "name"]
expertra_table = pd.concat([expertra_table, no_expertra_table], ignore_index = True)
                                   

banks_df = all_liquidated_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Moodys")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
moodys_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = moodys_table.append(one_bank_filled_merge)
moodys_table = pd.concat(moodys_table, ignore_index = True)
moodys_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
moodys_table.columns = ["DT", "Moodys", "name"]

banks_df = all_liquidated_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Moodys")]
no_moodys_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Moodys"] = "no"
    dates_template_local["name"] = bank_name
    no_moodys_table.append(dates_template_local)
no_moodys_table = pd.concat(no_moodys_table, ignore_index = True)
no_moodys_table.columns = ["DT", "Moodys", "name"]
moodys_table = pd.concat([moodys_table, no_moodys_table], ignore_index = True)



liquidated_ratings_table = moodys_table.merge(expertra_table, on = ["DT", "name"], how = "outer")
liquidated_results_table = liquidated_ratings_table.merge(all_liquidated_banks_data, left_on = "name", right_on = "short_name", how = "outer")

In [118]:
liquidated_results_table

,DT,Moodys,name,ExpertRA,short_name,form,regn,region,ownership,personal_page,app
0,2015-01-01,NaN,Бинбанк,NaN,Бинбанк,Публичное акционерное общество (ПАО),323,Москва,Государственный банк,0,1
1,2015-02-01,NaN,Бинбанк,NaN,Бинбанк,Публичное акционерное общество (ПАО),323,Москва,Государственный банк,0,1
2,2015-03-01,B2,Бинбанк,NaN,Бинбанк,Публичное акционерное общество (ПАО),323,Москва,Государственный банк,0,1
3,2015-04-01,B2,Бинбанк,NaN,Бинбанк,Публичное акционерное общество (ПАО),323,Москва,Государственный банк,0,1
4,2015-05-01,B3,Бинбанк,NaN,Бинбанк,Публичное акционерное общество (ПАО),323,Москва,Государственный банк,0,1
...,...,...,...,...,...,...,...,...,...,...,...
156022,2020-08-01,no,Яуза-банк,no,Яуза-банк,Товарищество с ограниченной ответственностью (...,2774,Москва,no,0,0
156023,2020-09-01,no,Яуза-банк,no,Яуза-банк,Товарищество с ограниченной ответственностью (...,2774,Москва,no,0,0
156024,2020-10-01,no,Яуза-банк,no,Яуза-банк,Товарищество с ограниченной ответственностью (...,2774,Москва,no,0,0
156025,2020-11-01,no,Яуза-банк,no,Яуза-банк,Товарищество с ограниченной ответственностью (...,2774,Москва,no,0,0


## Gather everything to one table

In [119]:
final = pd.concat([result_table, liquidated_results_table, withdrawn_results_table])

In [120]:
final

,DT,Moodys,name,ExpertRA,short_name,form,regn,region,ownership,personal_page,app
0,2015-01-01,no,Абсолют банк,no,Абсолют банк,Публичное акционерное общество (ПАО),2306,Москва,Частный банк,1,1
1,2015-02-01,no,Абсолют банк,A++,Абсолют банк,Публичное акционерное общество (ПАО),2306,Москва,Частный банк,1,1
2,2015-03-01,B1,Абсолют банк,A++,Абсолют банк,Публичное акционерное общество (ПАО),2306,Москва,Частный банк,1,1
3,2015-04-01,B1,Абсолют банк,A++,Абсолют банк,Публичное акционерное общество (ПАО),2306,Москва,Частный банк,1,1
4,2015-05-01,B1,Абсолют банк,A++,Абсолют банк,Публичное акционерное общество (ПАО),2306,Москва,Частный банк,1,1
...,...,...,...,...,...,...,...,...,...,...,...
40226,2020-08-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0
40227,2020-09-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0
40228,2020-10-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0
40229,2020-11-01,no,Ярослав,no,Ярослав,Закрытое акционерное общество (ЗАО),2906,Ярославская область,no,0,0


In [121]:
final.to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\Ratings.csv")

## Manually repeat for the banks which returned error

In [122]:
final = pd.read_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\Ratings.csv")

In [131]:
final[final.ownership == "Error"].name.unique()

array([], dtype=object)

In [9]:
final[final.name == "Дружба"]

,Unnamed: 0,DT,Moodys,name,ExpertRA,short_name,form,regn,region,ownership,personal_page,app
9903,9903,2015-01-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9904,9904,2015-02-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9905,9905,2015-03-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9906,9906,2015-04-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9907,9907,2015-05-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
...,...,...,...,...,...,...,...,...,...,...,...,...
9959,9959,2019-09-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9960,9960,2019-10-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9961,9961,2019-11-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error
9962,9962,2019-12-01,no,Дружба,no,Дружба,Error,Error,Error,Error,Error,Error


In [22]:
samples = sample_strings()
all_error_banks_data = [get_one_bank_data("https://www.bankodrom.ru/bank/druzhba/", samples = samples),
                  get_one_bank_data("https://www.bankodrom.ru/bank/ermak/", samples = samples),
                  get_one_bank_data("https://www.bankodrom.ru/bank/zapsibkombank/", samples = samples),
                  get_one_bank_data("https://www.bankodrom.ru/bank/mezhgosudarstvennyj-bank/", samples = samples),
                  get_one_bank_data("https://www.bankodrom.ru/bank/priobe/", samples = samples),
                  get_one_bank_data("https://www.bankodrom.ru/bank/strojlesbank/", samples = samples),
                  get_one_bank_data("https://www.bankodrom.ru/bank/surgutneftegazbank/", samples = samples)]
all_error_banks_data = pd.DataFrame(list(all_error_banks_data))

all_error_banks_links = ["https://www.bankodrom.ru/bank/druzhba/",
                        "https://www.bankodrom.ru/bank/ermak/",
                        "https://www.bankodrom.ru/bank/zapsipkombank/",
                        "https://www.bankodrom.ru/bank/mezhgosudarstvennyj-bank/",
                        "https://www.bankodrom.ru/bank/priobe/",
                        "https://www.bankodrom.ru/bank/strojlesbank/",
                        "https://www.bankodrom.ru/bank/surgutneftegazbank/"]
all_error_banks_names_urls = tuple(zip(all_error_banks_data.short_name, all_error_banks_links))
all_error_banks_moodys_list = list(pd.Series(all_error_banks_names_urls).apply(get_one_bank_credit_rating, samples = samples))
all_error_banks_moodys_table = pd.concat(all_error_banks_moodys_list, ignore_index = True)


Дружба
Ермак
Запсибкомбанк
Межгосударственный банк
Приобье
Стройлесбанк
Сургутнефтегазбанк


In [23]:
all_error_banks_moodys_table.name.unique()

array(['Дружба', 'Ермак', 'Запсибкомбанк', 'Межгосударственный банк',
       'Приобье', 'Стройлесбанк', 'Сургутнефтегазбанк'], dtype=object)

In [27]:
#moodys эти банки в общем то игнорировал
all_error_banks_moodys_table[(all_error_banks_moodys_table.rating!="no")&(all_error_banks_moodys_table.ratingname == "Moodys")]

,DT,rating,name,ratingname


In [28]:
dates_template = pd.DataFrame({"DateMonthly" : pd.date_range('01.01.2015','12.01.2020', 
              freq='MS').strftime("%d.%m.%Y").tolist()})
dates_template["DateMonthly"] = dates_template.DateMonthly.apply(lambda x: datetime.strptime(re.sub(" 00:00:00", "", x), "%Y-%m-%d").date())

banks_df = all_error_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT != "no")&(banks_df.ratingname == "Expertra")]
banks_df["DateMonthly"] = banks_df.DT.apply(lambda x: datetime.strptime(x, "%d.%m.%Y").date().replace(day = 1))
expertra_table = []
for bank_name in banks_df.name.unique():
    one_bank_df = banks_df[banks_df.name == bank_name]
    one_bank_empty_merge = dates_template.merge(one_bank_df, on = "DateMonthly", how = "left")
    one_bank_empty_merge = one_bank_empty_merge.sort_values(by = ["DateMonthly", "name", "DT"])
    one_bank_filled_merge = one_bank_empty_merge.fillna(method = "ffill")
    one_bank_filled_merge["name"] = bank_name
    one_bank_filled_merge = expertra_table.append(one_bank_filled_merge)
expertra_table = pd.concat(expertra_table, ignore_index = True)
expertra_table.drop(["DT", "ratingname"], axis = 1, inplace = True)
expertra_table.columns = ["DT", "ExpertRA", "name"]

banks_df = all_error_banks_moodys_table.copy()
banks_df = banks_df[(banks_df.DT == "no")&(banks_df.ratingname == "Expertra")]
no_expertra_table = []
for bank_name in banks_df.name.unique():
    dates_template_local = dates_template.copy()
    dates_template_local["Expertra"] = "no"
    dates_template_local["name"] = bank_name
    no_expertra_table.append(dates_template_local)
no_expertra_table = pd.concat(no_expertra_table, ignore_index = True)
no_expertra_table.columns = ["DT", "ExpertRA", "name"]
expertra_table = pd.concat([expertra_table, no_expertra_table], ignore_index = True)
                                   


error_ratings_table = expertra_table
error_results_table = error_ratings_table.merge(all_error_banks_data, left_on = "name", right_on = "short_name", how = "outer")

In [38]:
error_results_table[error_results_table.name == "Дружба"]

,DT,ExpertRA,name,short_name,form,regn,region,ownership,personal_page,app
184,2015-01-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
185,2015-02-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
186,2015-03-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
187,2015-04-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
188,2015-05-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
...,...,...,...,...,...,...,...,...,...,...
240,2019-09-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
241,2019-10-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
242,2019-11-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0
243,2019-12-01,no,Дружба,Дружба,Общество с ограниченной ответственностью (ООО),990,no,Частный банк,0,0


In [86]:
pd.DataFrame({"DateMonthly" : pd.date_range('01.01.2015','12.01.2020', 
              freq='MS').strftime("%d.%m.%Y").tolist()})

,DateMonthly
0,01.01.2015
1,01.02.2015
2,01.03.2015
3,01.04.2015
4,01.05.2015
...,...
67,01.08.2020
68,01.09.2020
69,01.10.2020
70,01.11.2020


In [34]:
final = final.drop(final.columns[0], axis = 1).append(error_results_table, ignore_index = True)

In [42]:
final = final[final.regn != "Error"]

In [43]:
final.to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\Ratings.csv")

#### Some preprocessing

In [132]:

ratings_df = pd.read_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\Ratings.csv")
ratings_df = ratings_df.drop(ratings_df.columns[0], axis = 1)
ratings_df.DT = pd.to_datetime(ratings_df.DT,format = "%Y-%m-%d")
ratings_df["Year"] = ratings_df.DT.apply(lambda x: x.year)
ratings_df["Month"] = ratings_df.DT.apply(lambda x: x.month)

In [133]:
ratings_df = ratings_df.fillna("no")
ratings_df.loc[ratings_df.ownership == "no", "ownership"] = "private"

In [134]:
ratings_df.ownership.unique()

array(['Частный банк', 'Государственный банк', 'Банк с госучастием',
       'Банк с иностранным капиталом', 'private'], dtype=object)

In [135]:
ratings_df.loc[ratings_df.ownership == "Банк с иностранным капиталом", "foreign"] = 1
ratings_df.loc[ratings_df.foreign != ratings_df.foreign, "foreign"] = 0


In [136]:
ratings_df.loc[ratings_df.ownership == "Частный банк", "ownership"] = "private"
ratings_df.loc[ratings_df.ownership == "Государственный банк", "ownership"] = "state"
ratings_df.loc[ratings_df.ownership == "Банк с госучастием", "ownership"] = "state"
ratings_df.loc[ratings_df.ownership == "Банк с иностранным капиталом", "ownership"] = "private"


In [137]:
ratings_df.form.unique()

array(['Публичное акционерное общество (ПАО)',
       'Общество с ограниченной ответственностью (ООО)',
       'Акционерное общество (АО)', 'no',
       'Открытое акционерное общество (ОАО)',
       'Акционерное общество закрытого типа (АОЗТ)',
       'Товарищество с ограниченной ответственностью (ТОО)',
       'Закрытое акционерное общество (ЗАО)',
       'Акционерное общество открытого типа (АООТ)'], dtype=object)

In [138]:
ratings_df.loc[ratings_df.form == "Публичное акционерное общество (ПАО)", "form"] = "public"
ratings_df.loc[ratings_df.form == 'Открытое акционерное общество (ОАО)', "form"] = "public"
ratings_df.loc[ratings_df.form == 'Акционерное общество открытого типа (АООТ)', "form"] = "public"
ratings_df.loc[ratings_df.form != 'public', "form"] = "private"


In [139]:
ratings_df.region.unique()

array(['Москва', 'Амурская область', 'Республика Татарстан (Татарстан)',
       'Санкт-Петербург', 'Удмуртская Республика', 'Приморский край',
       'Краснодарский край', 'Свердловская область',
       'Нижегородская область', 'Самарская область',
       'Костромская область', 'Пермский край', 'Ростовская область',
       'Саратовская область', 'Республика Мордовия',
       'Республика Саха (Якутия)', 'Алтайский край', 'Иркутская область',
       'Новосибирская область', 'Ивановская область',
       'Оренбургская область', 'Вологодская область',
       'Белгородская область', 'Ленинградская область',
       'Кемеровская область', 'Псковская область', 'Ульяновская область',
       'Астраханская область', 'Владимирская область',
       'Рязанская область', 'Республика Адыгея (Адыгея)',
       'Калужская область', 'Республика Крым', 'Тверская область',
       'Сахалинская область', 'no', 'Красноярский край', 'Омская область',
       'Республика Марий Эл', 'Курганская область',
       'Ре

In [140]:
ratings_df.loc[ratings_df.region != 'Москва', "region"] = "notmoscow"
ratings_df.loc[ratings_df.region == 'Москва', "region"] = "moscow"


In [141]:
ratings_df.drop("DT", axis = 1, inplace = True)

In [142]:
ratings_df.personal_page.unique()
ratings_df.app.unique()

array([1, 0], dtype=int64)

In [143]:
ratings_df.app = ratings_df.app.apply(str)
ratings_df.personal_page = ratings_df.personal_page.apply(str)


In [144]:
ratings_df.Moodys.unique()
ratings_df.ExpertRA.unique()

array(['no', 'A++', 'A+ (I)', 'A+ (III)', 'ruBBB', 'ruBBB-', 'ruBB+',
       'A+ (II)', 'A (II)', 'B+', 'B', 'ruCC', 'отозван', 'ruA-',
       'A (III)', 'ruB+', 'ruBB-', 'ruAA', '-', 'ruA', 'ruBB', 'A (I)',
       'B++', 'ruB-', 'ruAAA', 'ruAA+', 'ruBBB+', 'ruB', 'C++', 'C',
       'ruAA-', 'C+', 'ruCCC', 'ruA+', 'A+', 'E', 'ruC'], dtype=object)

In [145]:
ratings_df.loc[ratings_df.Moodys == "отозван", "Moodys"] = "withdrawn"
ratings_df.loc[ratings_df.ExpertRA == "отозван", "ExpertRA"] = "withdrawn"
ratings_df.loc[ratings_df.Moodys == "-", "Moodys"] = "no"
ratings_df.loc[ratings_df.ExpertRA == "-", "ExpertRA"] = "no"

In [146]:
ratings_df

,Moodys,name,ExpertRA,short_name,form,regn,region,ownership,personal_page,app,Year,Month,foreign
0,no,Абсолют банк,no,Абсолют банк,public,2306,moscow,private,1,1,2015,1,0.0
1,no,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,2,0.0
2,B1,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,3,0.0
3,B1,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,4,0.0
4,B1,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227886,no,Ярослав,no,Ярослав,private,2906,notmoscow,private,0,0,2020,8,0.0
227887,no,Ярослав,no,Ярослав,private,2906,notmoscow,private,0,0,2020,9,0.0
227888,no,Ярослав,no,Ярослав,private,2906,notmoscow,private,0,0,2020,10,0.0
227889,no,Ярослав,no,Ярослав,private,2906,notmoscow,private,0,0,2020,11,0.0


In [147]:
ratings_df.to_csv("C:\\Users\\Ivand\\Repos\\banks\\data\\RatingsClean.csv", index = False)

In [79]:
ratings_df

,Moodys,name,ExpertRA,short_name,form,regn,region,ownership,personal_page,app,Year,Month,foreign
0,no,Абсолют банк,no,Абсолют банк,public,2306,moscow,private,1,1,2015,1,0.0
1,no,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,2,0.0
2,B1,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,3,0.0
3,B1,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,4,0.0
4,B1,Абсолют банк,A++,Абсолют банк,public,2306,moscow,private,1,1,2015,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193082,no,Приобье,no,Приобье,private,537,notmoscow,private,0,0,2019,9,0.0
193083,no,Приобье,no,Приобье,private,537,notmoscow,private,0,0,2019,10,0.0
193084,no,Приобье,no,Приобье,private,537,notmoscow,private,0,0,2019,11,0.0
193085,no,Приобье,no,Приобье,private,537,notmoscow,private,0,0,2019,12,0.0
